In [1]:
# import necessary library
import pandas as pd
import re

/var/folders/jc/j5x1v8cd0fz9bck6fj4vf0280000gn/T/ipykernel_21027/581249992.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# read raw data of spotify review and remove unnecessary columns
data = pd.read_csv("SPOTIFY_REVIEWS.csv")
data = data.drop(columns=['Unnamed: 0'])
data

,review_id,pseudo_author_id,author_name,review_text,review_rating,review_likes,author_app_version,review_timestamp
0,14a011a8-7544-47b4-8480-c502af0ac26f,152618553977019693742,A Google user,Use it every day,5,1,1.1.0.91,2014-05-27 14:21:48
1,bfa8876b-470e-4640-83a7-77427f7f37e8,234382942865437071667,A Google user,"I enjoy the awesome UI of this app, and it has...",5,4,1.1.0.91,2014-05-27 14:36:02
2,70e8252f-058a-47d9-b066-df9e1571c970,174473604608358796368,A Google user,Love it! Especially the new design!,5,2,1.1.0.91,2014-05-27 14:40:01
3,672a155a-e81e-4d28-bdeb-a74c031bc072,286593453219054880269,A Google user,"Awesome UI, best music app out there!",5,1,1.1.0.91,2014-05-27 15:17:20
4,bbc1bf95-ed36-41a1-8b98-0f2e314caea5,167276875678680630145,A Google user,As a professional Android developer I'm glad t...,5,10,1.1.0.91,2014-05-27 15:26:48
...,...,...,...,...,...,...,...,...
3377418,41cafb32-c357-48df-97a3-e3867584f0d6,326345911143311484421,su************to,"Does not work, always says no internet but U t...",1,0,8.8.82.634,2023-11-15 23:11:02
3377419,ad68b04a-ff48-42f6-992b-bd765cdf9e94,322218874321066319125,Pe*******os,Goodjob,5,0,8.8.82.634,2023-11-15 23:11:29
3377420,9c36196d-12bc-4049-b7e4-27774a644bed,212802607434520579896,An*********ey,The blues always pull me through.,5,0,8.8.82.634,2023-11-15 23:11:32
3377421,ce45e1f8-78ad-40cc-a33b-d4e4672b8c78,415238355983315158902,St***en,awesome but you can't choose your song sometimes,4,0,8.8.82.634,2023-11-15 23:15:45


In [3]:
# cleaning function

# basic cleaning
# lower casing
# only keep alphabet and numeric
# remove extra spaces
def basic_cleaning(text):
    text = str(text).lower()
    pattern = r'[^a-zA-Z0-9\s]'
    text = re.sub(pattern, '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# there are some review that only contain numeric and don't have any information
# remove this kind of review
def remove_numeric_only_reviews(text):
    if re.match(r'^[\d\s]+$', text):
        return ''
    return text

# remove review with few alphabet, mostly don't contain any information
def remove_sparse_alphabetic_reviews(text, threshold=3):
    combined_text = text.replace(' ', '')
    if len(re.sub(r'[^a-zA-Z]', '', combined_text)) < threshold:
        return ''
    return text

In [4]:
# apply basic cleaning
data['cleaned_review_text'] = data['review_text'].apply(lambda x: basic_cleaning(x))

# filtering review with 5 words or more, 
# fewer word review might not contain usefull information
data['word_count'] = data['cleaned_review_text'].apply(lambda x: len(str(x).split()))
data = data[data['word_count'] > 4]

# apply more cleaning function
data['cleaned_review_text'] = data['cleaned_review_text'].apply(remove_numeric_only_reviews)
data['cleaned_review_text'] = data['cleaned_review_text'].apply(lambda x: remove_sparse_alphabetic_reviews(x, threshold=10))

# remove unusefull review
data['cleaned_review_text'].replace('', None, inplace=True)
data.dropna(subset=['cleaned_review_text'], inplace=True)

data

/var/folders/jc/j5x1v8cd0fz9bck6fj4vf0280000gn/T/ipykernel_21027/1973159113.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cleaned_review_text'] = data['cleaned_review_text'].apply(remove_numeric_only_reviews)
/var/folders/jc/j5x1v8cd0fz9bck6fj4vf0280000gn/T/ipykernel_21027/1973159113.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cleaned_review_text'] = data['cleaned_review_text'].apply(lambda x: remove_sparse_alphabetic_reviews(x, threshold=10))
/var/folders/jc/j5x1v8cd0fz9bck6fj4vf0

,review_id,pseudo_author_id,author_name,review_text,review_rating,review_likes,author_app_version,review_timestamp,cleaned_review_text,word_count
1,bfa8876b-470e-4640-83a7-77427f7f37e8,234382942865437071667,A Google user,"I enjoy the awesome UI of this app, and it has...",5,4,1.1.0.91,2014-05-27 14:36:02,i enjoy the awesome ui of this app and it has ...,18
2,70e8252f-058a-47d9-b066-df9e1571c970,174473604608358796368,A Google user,Love it! Especially the new design!,5,2,1.1.0.91,2014-05-27 14:40:01,love it especially the new design,6
3,672a155a-e81e-4d28-bdeb-a74c031bc072,286593453219054880269,A Google user,"Awesome UI, best music app out there!",5,1,1.1.0.91,2014-05-27 15:17:20,awesome ui best music app out there,7
4,bbc1bf95-ed36-41a1-8b98-0f2e314caea5,167276875678680630145,A Google user,As a professional Android developer I'm glad t...,5,10,1.1.0.91,2014-05-27 15:26:48,as a professional android developer im glad to...,26
5,eac4e85c-2e13-4626-9072-5e190a285cb5,279544562364680964711,A Google user,If I had to put a $ amount on how much I would...,5,4,1.1.0.91,2014-05-27 15:34:29,if i had to put a amount on how much i would h...,36
...,...,...,...,...,...,...,...,...,...,...
3377417,f758a94e-b824-445d-8011-0e40bf107f0d,180526216190456326659,Sj****05,"Bring back liked songs button, new add button ...",2,0,8.8.82.634,2023-11-15 23:06:56,bring back liked songs button new add button i...,31
3377418,41cafb32-c357-48df-97a3-e3867584f0d6,326345911143311484421,su************to,"Does not work, always says no internet but U t...",1,0,8.8.82.634,2023-11-15 23:11:02,does not work always says no internet but u tu...,11
3377420,9c36196d-12bc-4049-b7e4-27774a644bed,212802607434520579896,An*********ey,The blues always pull me through.,5,0,8.8.82.634,2023-11-15 23:11:32,the blues always pull me through,6
3377421,ce45e1f8-78ad-40cc-a33b-d4e4672b8c78,415238355983315158902,St***en,awesome but you can't choose your song sometimes,4,0,8.8.82.634,2023-11-15 23:15:45,awesome but you cant choose your song sometimes,8


In [6]:
# save cleaned text
data.to_csv("data/Cleaned Text.csv", index=False)